In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,Unnamed: 0,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,...,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef
0,0,0,NaN,152.0,426.0,0.0,0.0,0.0,0.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.627690,3.577721,2.198600
1,1,0,NaN,216.0,341.0,2.0,2.0,0.0,1.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608321,4.251999,6.533805
2,2,0,NaN,223.0,119.0,2.0,2.0,2.0,0.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166811,3.486596,3.821467
3,3,0,NaN,68.0,137.0,2.0,1.0,0.0,1.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.259244,3.592401,3.460103
4,4,1,NaN,425.0,225.0,2.0,2.0,1.0,0.0,-1,...,15.0,4.0,10.0,3.0,4.0,1.0,0.0,2.424278,3.414191,3.282876


In [4]:
df[df == -1] = np.nan

In [5]:
df['target'] = 0
df['target'][df['full_time_home_goals'] > df['full_time_away_goals']] = 1
df['target'][df['full_time_home_goals'] < df['full_time_away_goals']] = -1

<ipython-input-5-48f217678220>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][df['full_time_home_goals'] > df['full_time_away_goals']] = 1
<ipython-input-5-48f217678220>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][df['full_time_home_goals'] < df['full_time_away_goals']] = -1


In [6]:
Mean_enc_div = df.groupby(['Division'])['target'].mean().to_dict()
Mean_enc_home = df.groupby(['home_team'])['target'].mean().to_dict()
Mean_enc_away = df.groupby(['away_team'])['target'].mean().to_dict()

In [7]:
df['Division_mean'] = df['Division'].apply(lambda x: Mean_enc_div[x] if x in Mean_enc_div.keys() else 0.5)
df['home_team_mean'] = df['home_team'].apply(lambda x: Mean_enc_home[x] if x in Mean_enc_home.keys() else 0.5)
df['away_team_mean'] = df['away_team'].apply(lambda x: Mean_enc_away[x] if x in Mean_enc_away.keys() else 0.5)

In [8]:
df['home_team'] = df['home_team'].apply(lambda x: int(x))
df['away_team'] = df['away_team'].apply(lambda x: int(x))

In [9]:
df['buck_pred'] = np.argmin([df['away_coef'], df['draw_coef'], df['home_coef']], axis=0) - 1

In [10]:
dct = {team: [] for team in df['home_team'].unique()}
for team in df['home_team'].unique():
    dct[team].append(np.sum(df['full_time_home_goals'][df['home_team'] == team])
                     + np.sum(df['full_time_away_goals'][df['away_team'] == team]))
    dct[team].append((np.mean(df['home_shots'][df['home_team'] == team])
                     + np.mean(df['away_shots'][df['away_team'] == team])) / 2)
    dct[team].append((np.mean(df['home_fouls'][df['home_team'] == team])
                     + np.mean(df['away_fouls'][df['away_team'] == team])) / 2)
    dct[team].append((np.mean(df['home_yellow_cards'][df['home_team'] == team])
                     + np.mean(df['away_yellow_cards'][df['away_team'] == team])) / 2)
    dct[team].append((np.mean(df['home_red_cards'][df['home_team'] == team])
                     + np.mean(df['away_red_cards'][df['away_team'] == team])) / 2)

dct

{152: [180.0,
  11.3,
  13.766666666666666,
  2.1666666666666665,
  0.06666666666666667],
 216: [264.0,
  9.839285714285715,
  15.135757314974182,
  1.7885829030407345,
  0.0836201950659782],
 223: [233.0,
  13.658333333333333,
  15.59,
  2.1658333333333335,
  0.10166666666666666],
 68: [254.0,
  12.91875,
  13.414583333333333,
  2.1520833333333336,
  0.12708333333333333],
 425: [260.0,
  10.115707762557077,
  10.187671232876713,
  1.3739726027397259,
  0.1149771689497717],
 393: [277.0,
  9.347542997542998,
  11.642055692055692,
  1.8101146601146603,
  0.09516789516789516],
 239: [259.0,
  9.045659295659295,
  11.441072891072892,
  1.7610155610155611,
  0.08136773136773137],
 211: [306.0,
  10.666177908113392,
  13.032746823069404,
  1.8764051808406648,
  0.13685239491691104],
 99: [512.0,
  16.31718847560432,
  10.063776647935065,
  1.2756667558647756,
  0.050887521184550887],
 303: [185.0, nan, nan, nan, nan],
 75: [254.0,
  12.344223484848484,
  15.531723484848484,
  2.006628787878

In [12]:
df['home_sum_goals'] = df['home_team'].apply(lambda x: dct[x][0])
df['away_sum_goals'] = df['away_team'].apply(lambda x: dct[x][0])

df['home_mean_shots'] = df['home_team'].apply(lambda x: dct[x][1])
df['away_mean_shots'] = df['away_team'].apply(lambda x: dct[x][1])

df['home_mean_fouls'] = df['home_team'].apply(lambda x: dct[x][2])
df['away_mean_fouls'] = df['away_team'].apply(lambda x: dct[x][2])

df['home_mean_y_cards'] = df['home_team'].apply(lambda x: dct[x][3])
df['away_mean_y_cards'] = df['away_team'].apply(lambda x: dct[x][3])

df['home_mean_r_cards'] = df['home_team'].apply(lambda x: dct[x][4])
df['away_mean_r_cards'] = df['away_team'].apply(lambda x: dct[x][4])

In [13]:
df.head()

,Unnamed: 0,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,...,home_sum_goals,away_sum_goals,home_mean_shots,away_mean_shots,home_mean_fouls,away_mean_fouls,home_mean_y_cards,away_mean_y_cards,home_mean_r_cards,away_mean_r_cards
0,0,0,NaN,152,426,0.0,0.0,0.0,0.0,NaN,...,180.0,241.0,11.300000,11.731250,13.766667,11.125000,2.166667,1.666667,0.066667,0.127083
1,1,0,NaN,216,341,2.0,2.0,0.0,1.0,NaN,...,264.0,165.0,9.839286,13.352941,15.135757,14.235294,1.788583,1.500000,0.083620,0.088235
2,2,0,NaN,223,119,2.0,2.0,2.0,0.0,NaN,...,233.0,76.0,13.658333,NaN,15.590000,NaN,2.165833,NaN,0.101667,NaN
3,3,0,NaN,68,137,2.0,1.0,0.0,1.0,NaN,...,254.0,163.0,12.918750,13.795833,13.414583,13.177083,2.152083,1.954167,0.127083,0.093750
4,4,1,NaN,425,225,2.0,2.0,1.0,0.0,NaN,...,260.0,307.0,10.115708,10.059484,10.187671,11.982827,1.373973,1.749920,0.114977,0.125724


In [14]:
bef_features = ['away_team_mean', 'home_team_mean', 'Division_mean', 
                'Division', 'home_team', 'away_team', 'Referee',
                'home_coef', 'draw_coef', 'away_coef', 'buck_pred',
                'home_sum_goals', 'away_sum_goals', 'home_mean_shots',
                'away_mean_shots', 'home_mean_fouls', 'away_mean_fouls', 
                'home_mean_y_cards', 'away_mean_y_cards', 'home_mean_r_cards',
                'away_mean_r_cards']

In [22]:
X = X.fillna(X.median())

In [15]:
X = df[bef_features]
y = df['target']

In [23]:
X.head()

,away_team_mean,home_team_mean,Division_mean,Division,home_team,away_team,Referee,home_coef,draw_coef,away_coef,...,home_sum_goals,away_sum_goals,home_mean_shots,away_mean_shots,home_mean_fouls,away_mean_fouls,home_mean_y_cards,away_mean_y_cards,home_mean_r_cards,away_mean_r_cards
0,0.141414,0.072289,0.134712,0,152,426,49.0,3.627690,3.577721,2.198600,...,180.0,241.0,11.300000,11.731250,13.766667,11.125000,2.166667,1.666667,0.066667,0.127083
1,0.194030,0.240000,0.134712,0,216,341,49.0,1.608321,4.251999,6.533805,...,264.0,165.0,9.839286,13.352941,15.135757,14.235294,1.788583,1.500000,0.083620,0.088235
2,0.441176,-0.020408,0.134712,0,223,119,49.0,2.166811,3.486596,3.821467,...,233.0,76.0,13.658333,11.243802,15.590000,12.411765,2.165833,1.812500,0.101667,0.099155
3,0.238806,0.080808,0.134712,0,68,137,49.0,2.259244,3.592401,3.460103,...,254.0,163.0,12.918750,13.795833,13.414583,13.177083,2.152083,1.954167,0.127083,0.093750
4,0.099099,-0.037037,0.075700,1,425,225,49.0,2.424278,3.414191,3.282876,...,260.0,307.0,10.115708,10.059484,10.187671,11.982827,1.373973,1.749920,0.114977,0.125724


In [17]:
y

0        0
1        0
2        0
3        1
4        0
        ..
39987   -1
39988    1
39989   -1
39990    0
39991    0
Name: target, Length: 39992, dtype: int64

In [18]:
from catboost import Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier 

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
accuracy_score(y_pred, y_test)

0.4509313664208026

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cbc = CatBoostClassifier(iterations=3000, verbose=500)
cbc.fit(train_data)
y_pred = cbc.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
cbc = CatBoostClassifier(iterations=3000, verbose=500)

In [ ]:
cbc.fit(X_train, y_train, plot=True, 
        cat_features=['buck_pred', 'Division', 'home_team', 'away_team'])

In [ ]:
y_pred = cbc.predict(X_test)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test, y_test)
accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

In [ ]:
cbc = CatBoostClassifier(iterations=50, verbose=500)
cbc.fit(X, y)

In [ ]:
cbc.save_model('model')